# Agents: Multi-agents

- **Overview:** orchestrate multiple agents, tools, and models into a single workflow.  

- **Example:** Market Research Team

    Stepping into the role of a **technical AI lead at a fashion brand** preparing a summer sunglasses campaign, the task is to design a **fully automated creative pipeline** that mirrors a real-world business scenario.

- **List of agents:** 
    1. **Market Research Agent**
        - what’s trending in sunglasses fashion right now
        - cross-check those signals against your internal catalog
    2. **Graphic Designer Agent**
        - take the brief from your **Market Research Agent** and ask this one to translate it into a visual concept
    3. **Copywriter Agent**
        - creates elegant campaign quotes based on an image and a marketing trend summary
    4. **Packaging Agent**
        - take the trend summary, the campaign visual, the crafted quote, and the justification, and assemble them into an executive-ready markdown report

- **Steps:** 
    1. scans online sources for emerging fashion trends
    2. matches those trends with sunglasses in the internal catalog
    3. designs a campaign visual
    4. generates a short marketing quote
    5. finally packages everything into an **executive-ready report**.  

## 0. Initialization

In [1]:
# =========================
# Imports
# =========================

# --- Standard library ---
import base64
import json
import os
import re
from datetime import datetime
from io import BytesIO

# --- Third-party ---
import requests
import openai
from PIL import Image
from dotenv import load_dotenv
from IPython.display import Markdown, display
import aisuite

# --- Local / project ---
import tools
import utils


# =========================
# Environment & Client
# =========================
load_dotenv()
client = aisuite.Client()


## 1. Define tools  

### tavily_search_tool
`tools.tavily_search_tool`: performs live web searches to uncover evidence of current fashion trends.

In [ ]:
tools.tavily_search_tool('trends in sunglasses fashion')

[{'title': 'Top 10 Sunglasses Trends for Spring/Summer 2025 | Ultimate ...',
  'content': "Top 10 Sunglasses Trends for Spring/Summer 2025 | Ultimate Guide from MM Design\nMM Design\n36200 subscribers\n451 likes\n25493 views\n18 Mar 2025\nJoin Maria on mmDesign as she explores the top 10 sunglasses trends for Spring/Summer 2025. From classic Aviators to futuristic frames, discover styles featured in major designer shows like Loewe, Louis Vuitton, and Dior. Learn which frames suit different face shapes and how to integrate these trends into your wardrobe. Don't miss this comprehensive guide to stay fashionable this season!\n\n00:00 Introduction to Sunglasses Trends 2025\n00:40 Aviators: Timeless Cool\n01:38 Rounded Frames: Playful and Artistic\n02:31 Wayfarers: Classic Rebel\n03:28 Tinted Lenses: Making a Statement\n04:22 Wraparound Shields: Sporty and Futuristic\n05:14 Rectangle Frames: Sleek Minimalism\n06:17 Minimal Frames: Barely There Aesthetic\n07:01 Cat Eye Sunglasses: Feline Pow

### product_catalog_tool
`tools.product_catalog_tool`: which returns the internal sunglasses catalog. Each entry includes details such as product name, ID, description, stock quantity, and price. 

In [ ]:
tools.product_catalog_tool()

[{'name': 'Aviator',
  'item_id': 'SG001',
  'description': 'Originally designed for pilots, these teardrop-shaped lenses with thin metal frames offer timeless appeal. The large lenses provide excellent coverage while the lightweight construction ensures comfort during long wear.',
  'quantity_in_stock': 23,
  'price': 103},
 {'name': 'Wayfarer',
  'item_id': 'SG002',
  'description': 'Featuring thick, angular frames that make a statement, these sunglasses combine retro charm with modern edge. The rectangular lenses and sturdy acetate construction create a confident look.',
  'quantity_in_stock': 6,
  'price': 92},
 {'name': 'Mystique',
  'item_id': 'SG003',
  'description': 'Inspired by 1950s glamour, these frames sweep upward at the outer corners to create an elegant, feminine silhouette. The subtle curves and often embellished temples add sophistication to any outfit.',
  'quantity_in_stock': 3,
  'price': 88},
 {'name': 'Sport',
  'item_id': 'SG004',
  'description': 'Designed for 

## 2. Agent Definitions 
Define specialized agents that form a **marketing research team**

### 2.1. Market Research Agent  
- Output:
    - current fashion insights
- Steps:
    1. scans the web with `tavily_search_tool` and uncover what’s trending in sunglasses fashion right now
    2. cross-checks those signals against your internal catalog using `product_catalog_tool`, 
    3. returns a concise brief: the top fashion insights it found, the products that align with them, and a short explanation of why those picks make sense for your summer push. 

In [4]:
def market_research_agent(return_messages: bool = False):

    utils.log_agent_title_html("Market Research Agent", "🕵️‍♂️")

    prompt_ = f"""
You are a fashion market research agent tasked with preparing a trend analysis for a summer sunglasses campaign.

Your goal:
1. Explore current fashion trends related to sunglasses using web search.
2. Review the internal product catalog to identify items that align with those trends.
3. Recommend one or more products from the catalog that best match emerging trends.
4. If needed, today date is {datetime.now().strftime("%Y-%m-%d")}.

You can call the following tools:
- tavily_search_tool: to discover external web trends.
- product_catalog_tool: to inspect the internal sunglasses catalog.

Once your analysis is complete, summarize:
- The top 2–3 trends you found.
- The product(s) from the catalog that fit these trends.
- A justification of why they are a good fit for the summer campaign.
"""
    messages = [{"role": "user", "content": prompt_}]
    tools_ = tools.get_available_tools()

    while True:
        response = client.chat.completions.create(
            model="openai:o4-mini",
            messages=messages,
            tools=tools_,
            tool_choice="auto"
        )

        msg = response.choices[0].message

        if msg.content:
            utils.log_final_summary_html(msg.content)
            return (msg.content, messages) if return_messages else msg.content

        if msg.tool_calls:
            for tool_call in msg.tool_calls:
                utils.log_tool_call_html(tool_call.function.name, tool_call.function.arguments)
                result = tools.handle_tool_call(tool_call)
                utils.log_tool_result_html(result)

                messages.append(msg)
                messages.append(tools.create_tool_response_message(tool_call, result))
        else:
            utils.log_unexpected_html()
            return ("[⚠️ Unexpected: No tool_calls or content returned]", messages) if return_messages else "[⚠️ Unexpected: No tool_calls or content returned]"

In [5]:
market_research_result = market_research_agent()

### 2.2. Graphic Designer Agent  
- Input:
  - fashion insights
- Output:
  - campaign image + caption
- Graphic Designer Agent takes the brief from Market Research Agent and translate it into a visual concept.  
- Steps:  
  1. uses `aisuite` with an OpenAI text model (`o4-mini`) to craft a vivid **prompt** and a short, engaging **caption**.  
  2. sends the prompt to OpenAI’s `dall-e-3` API to generate the **campaign image** itself.  
- Result: the generated image (saved locally for reuse), the exact prompt that produced it (useful for iteration), and a polished caption for campaign storytelling.  

In [6]:
def graphic_designer_agent(trend_insights: str, caption_style: str = "short punchy", size: str = "1024x1024") -> dict:

    """
    Uses aisuite to generate a marketing prompt/caption and OpenAI (directly) to generate the image.

    Args:
        trend_insights (str): Trend summary from the researcher agent.
        caption_style (str): Optional style hint for caption.
        size (str): Image resolution (e.g., '1024x1024').

    Returns:
        dict: A dictionary with image_url, prompt, and caption.
    """

    utils.log_agent_title_html("Graphic Designer Agent", "🎨")

    # Step 1: Generate prompt and caption using aisuite
    system_message = (
        "You are a visual marketing assistant. Based on the input trend insights, "
        "write a creative and visual prompt for an AI image generation model, and also a short caption."
    )

    user_prompt = f"""
Trend insights:
{trend_insights}

Please output:
1. A vivid, descriptive prompt to guide image generation.
2. A marketing caption in style: {caption_style}.

Respond in this format:
{{"prompt": "...", "caption": "..."}}
"""

    chat_response = client.chat.completions.create(
        model="openai:o4-mini",
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_prompt}
        ]
    )

    content = chat_response.choices[0].message.content.strip()
    match = re.search(r'\{.*\}', content, re.DOTALL)
    parsed = json.loads(match.group(0)) if match else {"error": "No JSON returned", "raw": content}

    prompt = parsed["prompt"]
    caption = parsed["caption"]

    # Step 2: Generate image directly using openai-python
    openai_client = openai.OpenAI()

    image_response = openai_client.images.generate(
        model="dall-e-3",
        prompt=prompt,
        size=size,
        quality="standard",
        n=1,
        response_format="url"
    )

    image_url = image_response.data[0].url

    # Save image locally
    img_bytes = requests.get(image_url).content
    img = Image.open(BytesIO(img_bytes))

    filename = os.path.basename(image_url.split("?")[0])
    image_path = filename
    img.save(image_path)


    # Log summary with local image
    utils.log_final_summary_html(f"""
        <h3>Generated Image and Caption</h3>

        <p><strong>Image Path:</strong> <code>{image_path}</code></p>

        <p><strong>Generated Image:</strong></p>
        <img src="{image_path}" alt="Generated Image" style="max-width: 100%; height: auto; border: 1px solid #ccc; border-radius: 8px; margin-top: 10px; margin-bottom: 10px;">

        <p><strong>Prompt:</strong> {prompt}</p>
    """)


    return {
        "image_url": image_url,
        "prompt": prompt,
        "caption": caption,
        "image_path": image_path  
    }



In [ ]:
graphic_designer_agent_result = graphic_designer_agent(
    trend_insights=market_research_result,
)

### 2.3. Copywriter Agent  
- Input:
    - fashion insights
    - campaign image
- Output:
    - campaign quote + justification
- **Copywriter Agent** takes the visual and the analysis together as multimodal input and crafts a short, elegant marketing quote that captures the essence of the message. 
- Along with the quote, it gives a clear justification—why the phrase fits the image and how it ties back to the trends. This way, you don’t just get a catchy line, you also get the reasoning behind it, making it easier to defend and refine in front of stakeholders.  

In [8]:
def copywriter_agent(image_path: str, trend_summary: str, model: str = "openai:o4-mini") -> dict:

    """
    Uses aisuite (OpenAI only) to send an image and a trend summary and return a campaign quote.

    Args:
        image_path (str): URL of the image to be analyzed.
        trend_summary (str): Text from the researcher agent.
        model (str): OpenAI model (e.g., openai:o4-mini, openai:gpt-4o)

    Returns:
        dict: {
            "quote": "...",
            "justification": "...",
            "image_path": "..."
        }
    """

    utils.log_agent_title_html("Copywriter Agent", "✍️")

    # Step 1: Load local image and encode as base64
    with open(image_path, "rb") as f:
        img_bytes = f.read()

    b64_img = base64.b64encode(img_bytes).decode("utf-8")

    # Step 2: Build OpenAI-compliant multimodal message
    messages = [
        {
            "role": "system",
            "content": "You are a copywriter that creates elegant campaign quotes based on an image and a marketing trend summary."
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/png;base64,{b64_img}",
                        "detail": "auto"
                    }
                },
                {
                    "type": "text",
                    "text": f"""
Here is a visual marketing image and a trend analysis:

Trend summary:
\"\"\"{trend_summary}\"\"\"

Please return a JSON object like:
{{
  "quote": "A short, elegant campaign phrase (max 12 words)",
  "justification": "Why this quote matches the image and trend"
}}"""
                }
            ]
        }
    ]

    # Step 3: Send request via aisuite
    response = client.chat.completions.create(
        model=model,
        messages=messages,
    )

    # Step 4: Parse JSON response
    content = response.choices[0].message.content.strip()

    utils.log_final_summary_html(content)

    try:
        match = re.search(r'\{.*\}', content, re.DOTALL)
        parsed = json.loads(match.group(0)) if match else {"error": "No valid JSON returned"}
    except Exception as e:
        parsed = {"error": f"Failed to parse: {e}", "raw": content}


    parsed["image_path"] = image_path
    return parsed


In [9]:
copywriter_agent_result = copywriter_agent(
    image_path=graphic_designer_agent_result["image_path"],
    trend_summary=market_research_result,
)

### 2.4. Packaging Agent  

- **Packaging Agent** ties everything together. After the **Market Research Agent**, **Graphic Designer Agent**, and **Copywriter Agent** have each contributed their part, this agent compiles the entire story into one polished artifact.  
- Input:
    - trend summary
    - the campaign visual
    - the crafted quote + justification,
- Output:
    - an executive-ready markdown report. 
Along the way, it rewrites the trend insights for clarity and tone, ensures the quote is styled properly with the image, and organizes everything so the final document looks professional and persuasive.  

- With this step, you end up with a complete campaign package—easy to share, visually engaging, and ready for CEO-level review. 

- The final result includes:  
    - A refined trend summary that you will see rewritten for executive clarity  
    - A visually styled image with your campaign quote overlaid using HTML  
    - A clear justification so you understand why the visual and message align with current trends  
    - A timestamp showing you exactly when the report was generated  

In [10]:
def packaging_agent(trend_summary: str, image_url: str, quote: str, justification: str, output_path: str = "campaign_summary.md") -> str:

    """
    Packages the campaign assets into a beautifully formatted markdown report for executive review.

    Args:
        trend_summary (str): Summary of the market trends.
        image_url (str): URL of the campaign image.
        quote (str): Marketing quote to overlay.
        justification (str): Explanation for the quote.
        output_path (str): Path to save the markdown report.

    Returns:
        str: Path to the saved markdown file.
    """

    utils.log_agent_title_html("Packaging Agent", "📦")

    # We use this path in the src of the <img>
    styled_image_html = f"""
![Open the generated file to see]({image_url})
    """

    beautified_summary = client.chat.completions.create(
        model="openai:o4-mini",
        messages=[
            {"role": "system", "content": "You are a marketing communication expert writing elegant campaign summaries for executives."},
            {"role": "user", "content": f"""
Please rewrite the following trend summary to be clear, professional, and engaging for a CEO audience:

\"\"\"{trend_summary.strip()}\"\"\"
"""}
        ]
    ).choices[0].message.content.strip()

    utils.log_tool_result_html(beautified_summary)

    # Combine all parts into markdown
    markdown_content = f"""# 🕶️ Summer Sunglasses Campaign – Executive Summary

## 📊 Refined Trend Insights
{beautified_summary}

## 🎯 Campaign Visual
{styled_image_html}

## ✍️ Campaign Quote
{quote.strip()}

## ✅ Why This Works
{justification.strip()}

---

*Report generated on {datetime.now().strftime('%Y-%m-%d')}*
"""

    with open(output_path, "w", encoding="utf-8") as f:
        f.write(markdown_content)

    return output_path



In [11]:
packaging_agent_result = packaging_agent(
    trend_summary=market_research_result,
    image_url=graphic_designer_agent_result["image_path"],
    quote=copywriter_agent_result["quote"],
    justification=copywriter_agent_result["justification"],
    output_path=f"campaign_summary_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.md"
)

In [ ]:
# Load and render the markdown content
with open(packaging_agent_result, "r", encoding="utf-8") as f:
    md_content = f.read()

display(Markdown(md_content))

# 🕶️ Summer Sunglasses Campaign – Executive Summary

## 📊 Refined Trend Insights
Executive Summary  
As we prepare to launch our Summer 2025 sunglasses campaign, three powerful style currents will define the season—and our core assortment of SKUs positions us to capture each one with maximum impact:

1. Statement-Tinted Lenses  
   • Why it matters: Bold colors and soft pastels are driving social buzz and broadening appeal beyond traditional sunglass shoppers.  
   • Product fit: SG005 “Round” (on-hand: 10 units) – Its minimalist circular frame becomes a canvas for eye-catching pastel gradients or dual-tone lenses, tapping directly into consumer desire for personalization and self-expression.

2. Sporty-Futuristic Shields  
   • Why it matters: Performance-inspired, single-lens shields with tech-driven details are migrating from athletic niches into mainstream streetwear.  
   • Product fit: SG004 “Sport” (on-hand: 11 units) – With its wraparound profile and rubberized temple grips, this model bridges functionality and fashion, appealing to the athleisure and urban markets alike.

3. Retro-Classic Revivals  
   • Why it matters: Timeless silhouettes (Aviator, Wayfarer, Cat-Eye) are experiencing a modern renaissance, fueled by oversized proportions, mixed materials and statement hardware.  
   • Product fit: SG001 “Aviator” (on-hand: 23 units) – The iconic teardrop shape enjoys renewed momentum; offered in classic smoke or new tinted finishes, it can be accessorized with a detachable chain for lifestyle differentiation.

Strategic Recommendations  
• Lead with “Round” in color-driven hero imagery—highlight pastel and gradient lens treatments across digital, social and in-store displays.  
• Showcase “Sport” in dynamic athletic and urban environments, underscoring its wind-protection and futuristic aesthetic in video and influencer content.  
• Position “Aviator” as the “everyday hero” style—feature both heritage (smoke lenses) and trend-forward (tint, chain accessory) executions to maximize cross-trend appeal.  

By centering our campaign on these three SKUs, we deliver a targeted yet diversified offering that speaks directly to summer’s most viral style moments—driving engagement, elevating brand perception and ultimately accelerating sell-through and margin performance.

## 🎯 Campaign Visual

![Open the generated file to see](img-otTDbYm6FWiw0y3nC8ieYpw6.png)
    

## ✍️ Campaign Quote
Chase sunsets in bold pastel tints.

## ✅ Why This Works
This phrase evokes the active seaside energy shown in the image and highlights summer’s key trend of colorful, gradient lenses—tying together the sporty-futuristic drive and the timeless appeal of statement sunglasses.

---

*Report generated on 2025-12-11*


## 3. End-to-end Pipeline – `run_sunglasses_campaign_pipeline`

The function will:  
- Run market research to scan fashion trends and match them to your catalog.  
- Generate a visually styled image and caption.  
- Create a short, elegant campaign quote with justification.  
- Package everything into a polished markdown report tailored for executive review.  

In [13]:
def run_sunglasses_campaign_pipeline(output_path: str = "campaign_summary.md") -> dict:
    """
    Runs the full summer sunglasses campaign pipeline:
    1. Market research (search trends + match products)
    2. Generate visual + caption
    3. Generate quote based on image + trend
    4. Create executive markdown report

    Returns:
        dict: Dictionary containing all intermediate results + path to final report
    """
    # 1. Run market research agent
    trend_summary = market_research_agent()
    print("✅ Market research completed")

    # 2. Generate image + caption
    visual_result = graphic_designer_agent(trend_insights=trend_summary)
    image_path = visual_result["image_path"]
    print("🖼️ Image generated")

    # 3. Generate quote based on image + trends
    quote_result = copywriter_agent(image_path=image_path, trend_summary=trend_summary)
    quote = quote_result.get("quote", "")
    justification = quote_result.get("justification", "")
    print("💬 Quote created")

    # 4. Generate markdown report
    md_path = packaging_agent(
        trend_summary=trend_summary,
        image_url=image_path,  
        quote=quote,
        justification=justification,
        output_path=f"campaign_summary_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.md"
    )

    print(f"📦 Report generated: {md_path}")

    return {
        "trend_summary": trend_summary,
        "visual": visual_result,
        "quote": quote_result,
        "markdown_path": md_path
    }


You can now create a complete campaign report by running the pipeline in a single call. Just execute the next cell:  

In [14]:
results = run_sunglasses_campaign_pipeline()

✅ Market research completed


🖼️ Image generated


💬 Quote created


📦 Report generated: campaign_summary_2025-12-11_03-34-07.md


In [ ]:
with open(results["markdown_path"], "r", encoding="utf-8") as f:
    md_content = f.read()
display(Markdown(md_content))

# 🕶️ Summer Sunglasses Campaign – Executive Summary

## 📊 Refined Trend Insights
Executive Summary  
As we prepare to launch our Summer 2025 sunglasses campaign, three powerful style currents will define the season—and our core assortment of SKUs positions us to capture each one with maximum impact:

1. Statement-Tinted Lenses  
   • Why it matters: Bold colors and soft pastels are driving social buzz and broadening appeal beyond traditional sunglass shoppers.  
   • Product fit: SG005 “Round” (on-hand: 10 units) – Its minimalist circular frame becomes a canvas for eye-catching pastel gradients or dual-tone lenses, tapping directly into consumer desire for personalization and self-expression.

2. Sporty-Futuristic Shields  
   • Why it matters: Performance-inspired, single-lens shields with tech-driven details are migrating from athletic niches into mainstream streetwear.  
   • Product fit: SG004 “Sport” (on-hand: 11 units) – With its wraparound profile and rubberized temple grips, this model bridges functionality and fashion, appealing to the athleisure and urban markets alike.

3. Retro-Classic Revivals  
   • Why it matters: Timeless silhouettes (Aviator, Wayfarer, Cat-Eye) are experiencing a modern renaissance, fueled by oversized proportions, mixed materials and statement hardware.  
   • Product fit: SG001 “Aviator” (on-hand: 23 units) – The iconic teardrop shape enjoys renewed momentum; offered in classic smoke or new tinted finishes, it can be accessorized with a detachable chain for lifestyle differentiation.

Strategic Recommendations  
• Lead with “Round” in color-driven hero imagery—highlight pastel and gradient lens treatments across digital, social and in-store displays.  
• Showcase “Sport” in dynamic athletic and urban environments, underscoring its wind-protection and futuristic aesthetic in video and influencer content.  
• Position “Aviator” as the “everyday hero” style—feature both heritage (smoke lenses) and trend-forward (tint, chain accessory) executions to maximize cross-trend appeal.  

By centering our campaign on these three SKUs, we deliver a targeted yet diversified offering that speaks directly to summer’s most viral style moments—driving engagement, elevating brand perception and ultimately accelerating sell-through and margin performance.

## 🎯 Campaign Visual

![Open the generated file to see](img-otTDbYm6FWiw0y3nC8ieYpw6.png)
    

## ✍️ Campaign Quote
Chase sunsets in bold pastel tints.

## ✅ Why This Works
This phrase evokes the active seaside energy shown in the image and highlights summer’s key trend of colorful, gradient lenses—tying together the sporty-futuristic drive and the timeless appeal of statement sunglasses.

---

*Report generated on 2025-12-11*
